In [46]:
import pymysql
import pandas as pd
import warnings
import sys
from datetime import datetime
warnings.filterwarnings('ignore')

In [47]:
def connect_mysql():
    conn = pymysql.connect(
        host = '127.0.0.1', # 连接名称，默认127.0.0.1
        user = 'root', # 用户名
        passwd='2020Cnic@!', # 密码
        port= 3306, # 端口，默认为3306
        db='oj', # 数据库名称
        charset='utf8', # 字符编码
    )
    return conn

In [48]:
def show_table_content(cur, table_name:str):
    sql = "SELECT * FROM `%s`" % table_name
    
    cur.execute(sql) # 执行SQL语句
    data = cur.fetchall() # 通过fetchall方法获得数据
    cur.execute("SHOW COLUMNS FROM `%s`" % table_name)
    head = cur.fetchall()
    print(head)
    for d in data:
        print(d)

In [49]:
def show_table_list(cur):
    tables_list = []
    data = cur.execute("show tables")
    data = cur.fetchall()
    for d in data:
        print(d[0])
    return tables_list

In [50]:
def get_user_id_dict(cur, id_range:tuple):
    name_id_dict = {}
    min_id, max_id = id_range
    # User key:
    ## 0. id
    ## 1. 2020K...
    ## 2. name 
    
    cur.execute("SELECT * FROM User")
    data = cur.fetchall()

    for d in data:
        if min_id <= int(d[0]) and max_id >= int(d[0]):
            name_id_dict[d[2]] = d[0]
    return name_id_dict 

In [51]:
def get_exp_id_dict(cur, id_range:tuple = (0,1000)):
    # User key:
    ## 0. id
    ## 1. socket应用
    exp_id_dict = {}
    min_id, max_id = id_range
    
    cur.execute("SELECT * FROM Contest")
    data = cur.fetchall()
    for d in data:
        if min_id <= int(d[0]) and max_id >= int(d[0]):
            exp_id_dict[d[1]] = d[0]
    return exp_id_dict

In [52]:
def late_submission(time:str, ddl:str):
    format_pattern = "%Y-%m-%d %H:%M:%S"
    difference = (datetime.strptime(time, format_pattern) - datetime.strptime(ddl, format_pattern))
    
    if difference.days >= 0:
        return True
    else:
        return False

In [53]:
def get_score_from_res(res, ddl_time):
    res = list(res)
    name_id = res[2]
    exp_id  = res[1]
    status  = res[4]
    desc    = res[5]
    uptime  = str(res[-1])
    if "编译失败" in desc or "正在判题" in desc:
        return 0
    standard_score = round(10 * desc.count("Pass") / (desc.count("Pass") + desc.count("Fail")),2)
    if late_submission(uptime, ddl_time):
        standard_score *= 0.8

    return standard_score
    

In [54]:
def get_stu_exp_score(cur, name:str, name_id:int, exp:str, exp_id:int, ddl_time:str):
    cur.execute("SELECT * FROM FileUploadRecord WHERE userId = %d AND expId = %d" % (name_id, exp_id))
    data = cur.fetchall()
    
    # 完全未提交时，为0分
    if not data:
        return 0 # no submit, score 0
    else:
        last_submit_res = list(data)[-1]
        score = get_score_from_res(last_submit_res, ddl_time)
    #sys.exit(0)
    return score

In [55]:
def generate_score(cur, name_id_dict:dict, exp_id_dict:dict, exp_info_dict:dict):
    index = list(name_id_dict.keys())
    columns = list(exp_id_dict.keys())
    df = pd.DataFrame(columns=columns, index=index)
    
    #print(index,columns)
    for name, name_id in name_id_dict.items():
        for exp, exp_id in exp_id_dict.items():
            print(name, name_id, exp, exp_id)
            if int(name_id) != -1 and int(exp_id) != -1:
                df.loc[name, exp] = get_stu_exp_score(cur, name, name_id, exp, exp_id, exp_info_dict[exp_id]["ddl_time"])
    return df

In [56]:
def add_absence_info(df:pd.DataFrame, name_id_dict:dict, filename:str):
    # the content of filename is copy from sep
    with open(filename, "r") as f:
        lines = f.readlines()
        lines = [line.strip() for line in lines]

    item_count = {}
    for line in lines:
        items = line.split()
        for item in items:
            if item in item_count:
                item_count[item] += 1
            else:
                item_count[item] = 1
    absence_list = []
    for name in list(name_id_dict.keys()):
        absence_list.append(item_count[name] if name in list(item_count.keys()) else 0)
    df["absence"] = absence_list
    print(absence_list)

In [57]:
def del_user_id_dict(cur, lowest_stu_id = 200):
    cur.execute("use oj")
    cur.execute("DELETE from User WHERE userId>=%d" % lowest_stu_id)
    cur.execute("SELECT * FROM User")
    data = cur.fetchall()
    for d in data:
        print(d)

In [58]:
#连接数据库
conn = connect_mysql()
cur = conn.cursor() # 生成游标对象
#show_table_list(cur)
#show_table_content(cur, "FileUploadRecord")
term_start = True
if term_start:
    # after deleting, restarting docker is necessary
    del_user_id_dict(cur)
else:
    name_id_dict = get_user_id_dict(cur, (429,455))
    exp_id_dict  = get_exp_id_dict(cur, (12,24))# only consider this term's submit
    exp_info_list = {
        13:{"exp_name": "socket",    "ddl_time": "2024-04-03 23:59:59"},
        14:{"exp_name": "hub",       "ddl_time": "2024-04-10 23:59:59"},
        15:{"exp_name": "switch",    "ddl_time": "2024-04-10 23:59:59"},
        16:{"exp_name": "stp",       "ddl_time": "2024-04-17 23:59:59"},
        17:{"exp_name": "router",    "ddl_time": "2024-05-08 23:59:59"},
        18:{"exp_name": "mospf",     "ddl_time": "2024-05-22 23:59:59"},
        19:{"exp_name": "ip_lookup", "ddl_time": "2024-05-29 23:59:59"},
        20:{"exp_name": "echo",      "ddl_time": "2024-06-12 23:59:59"},
        21:{"exp_name": "bulk",      "ddl_time": "2024-06-12 23:59:59"},
        22:{"exp_name": "nat",       "ddl_time": "2024-06-18 23:59:59"},
        23:{"exp_name": "bulk_loss", "ddl_time": "2024-06-22 23:59:59"},
        24:{"exp_name": "graft",     "ddl_time": "2024-06-26 23:59:59"},
    }
    # get score
    df = generate_score(cur, name_id_dict, exp_id_dict, exp_info_list)
    # add absence info
    add_absence_info(df, name_id_dict, "缺勤名单.txt")

    df.to_csv("score.csv")
    df.to_excel("score.xlsx")
cur.close() # 关闭游标
conn.close() # 关闭连接

(8, 'lyg', '梁阳光', '95b0995957ff5fefaa7e0ff453dc8284', 0, None, datetime.datetime(2021, 8, 13, 13, 43, 46), datetime.datetime(2024, 8, 13, 11, 25, 31), None, None, None, None, 0, 0)
(130, 'yjb', '杨金宝', '95b0995957ff5fefaa7e0ff453dc8284', 0, None, datetime.datetime(2021, 8, 24, 10, 35, 33), datetime.datetime(2022, 3, 30, 18, 45, 45), None, None, None, None, 0, 0)
